In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, auc, accuracy_score
from sklearn.preprocessing import StandardScaler, LabelEncoder
from tensorflow.keras.layers import *
from tensorflow.keras import models, Model
from tensorflow.keras.initializers import RandomNormal
import qkeras
from qkeras import *
from sparsepixels.layers import *
from sparsepixels.utils import *
import h5py
import glob
import os

In [ ]:
def build_dummy():
    quantizer = quantized_bits(6, 0, alpha=1)

    x_in = keras.Input(shape=(10, 20, 1), name='x_in')
    x = QConv2D(filters=2, kernel_size=(3, 3), use_bias=True, name='conv1',
                padding='same', strides=1,
                kernel_quantizer=quantizer, bias_quantizer=quantizer,
                kernel_initializer=RandomNormal(mean=0, stddev=1),
                bias_initializer=RandomNormal(mean=0, stddev=1))(x_in)

    return keras.Model(x_in, x, name='dummy')

model_dummy = build_dummy()
model_dummy.compile(optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=0.001),
                    loss='categorical_crossentropy', metrics = ['accuracy'])
print(model_dummy.summary())

In [ ]:
import hls4ml
config = hls4ml.utils.config_from_keras_model(model_dummy, granularity='name', backend='Vitis')
config

model_dummy_hls = hls4ml.converters.convert_from_keras_model(
    model_dummy,
    hls_config=config,
    project_name='hls_dummy',
    backend='Vitis',
    output_dir='hls_proj/scaling/dummy',
    part='xcu250-figd2104-2L-e',
    #io_type='io_stream',
    io_type='io_parallel',
)

#model_dummy_hls.compile()
model_dummy_hls.write()

In [3]:
import numpy as np
numbers = np.random.rand(100) * 2
print(' '.join(f'{x:.4g}' for x in numbers))

1.265 1.48 1.501 1.091 1.373 0.6441 0.7005 1.392 0.4515 0.7102 1.558 1.189 0.97 1.618 0.127 0.2424 1.255 1.848 0.5944 1.821 1.356 0.4194 0.6264 1.448 0.8749 0.6752 0.8987 0.3346 0.9824 1.527 0.9075 0.6086 0.2413 1.807 0.9041 0.2382 1.88 1.693 1.821 0.3723 1.177 1.066 0.03106 1.08 0.02853 1.326 1.491 1.233 1.862 0.9069 0.7599 1.098 1.232 1.847 1.878 0.943 0.05972 1.885 1.764 1.451 1.913 1.604 1.306 1.637 1.893 0.5005 0.6056 1.924 0.7432 0.7951 0.1448 1.407 1.064 1.146 1.933 1.21 1.915 1.052 1.211 0.4339 0.7145 1.626 0.9511 0.2439 0.1919 1.462 1.779 0.7929 0.7994 1.682 1.677 1.436 1.191 0.1551 0.9829 1.724 0.337 1.364 0.1855 0.9675
